In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

D:\alex_space\software\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\alex_space\software\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [6]:
def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1, 1),
                        min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()
    
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")
    
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

In [8]:
book_data = pd.read_csv('../../../chapter-9/cluster/data/data.csv')
print(book_data.head())

book_titles = book_data['title'].tolist()
book_content = book_data['content'].tolist()

print('书名:', book_titles[0])
print('内容:', book_content[0][:10])

    title         tag                                              info  \
0  ﻿解忧杂货店  豆瓣图书标签: 小说         [日] 东野圭吾 / 李盈春 / 南海出版公司 / 2014-5 / 39.50元   
1   巨人的陨落  豆瓣图书标签: 小说  [英] 肯·福莱特 / 于大卫 / 江苏凤凰文艺出版社 / 2016-5-1 / 129.80元   
2   我的前半生  豆瓣图书标签: 小说                     亦舒 / 新世界出版社 / 2007-8 / 22.00元   
3    百年孤独  豆瓣图书标签: 小说   [哥伦比亚] 加西亚·马尔克斯 / 范晔 / 南海出版公司 / 2011-6 / 39.50元   
4   追风筝的人  豆瓣图书标签: 小说     [美] 卡勒德·胡赛尼 / 李继宏 / 上海人民出版社 / 2006-5 / 29.00元   

      comments                                            content  
0  (225675人评价)  现代人内心流失的东西，这家杂货店能帮你找回——\r\n僻静的街道旁有一家杂货店，只要写下烦恼...  
1   (22536人评价)  在第一次世界大战的硝烟中，每一个迈向死亡的生命都在热烈地生长——威尔士的矿工少年、刚失恋的美...  
2   (20641人评价)  一个三十几岁的美丽女人子君，在家做全职家庭主妇。却被一个平凡女人夺走丈夫，一段婚姻的失败，让...  
3  (111883人评价)  《百年孤独》是魔幻现实主义文学的代表作，描写了布恩迪亚家族七代人的传奇故事，以及加勒比海沿岸...  
4  (278905人评价)  12岁的阿富汗富家少爷阿米尔与仆人哈桑情同手足。然而，在一场风筝比赛后，发生了一件悲惨不堪的...  
书名: ﻿解忧杂货店
内容: 现代人内心流失的东西


In [10]:
from normalization import normalize_corpus

# normalize corpus
norm_book_content = normalize_corpus(book_content)

# 提取 tf-idf 特征
vectorizer, feature_matrix = build_feature_matrix(norm_book_content,
                                                  feature_type='tfidf',
                                                  min_df=0.2, max_df=0.90,
                                                  ngram_range=(1, 2))
# 查看特征数量
print(feature_matrix.shape)

# 获取特征名字
feature_names = vectorizer.get_feature_names()

# 打印某些特征
print(feature_names[:10])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Alex\AppData\Local\Temp\jieba.cache
Loading model cost 0.634 seconds.
Prefix dict has been built succesfully.


(2822, 16281)
['000', '01', '07', '09', '10', '100', '1000', '1001', '108758', '11']


In [11]:
from sklearn.cluster import KMeans

def k_means(feature_matrix, num_clusters=10):
    km = KMeans(n_clusters=num_clusters, max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

num_clusters = 10
km_obj, clusters = k_means(feature_matrix=feature_matrix, num_clusters=num_clusters)
book_data['Cluster'] = clusters

D:\alex_space\software\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [12]:
from collections import Counter

# 获取每个cluster的数量
c = Counter(clusters)
print(c.items())

def get_cluster_data(clustering_obj, book_data, feature_names, num_clusters, topn_features=10):
    cluster_details = {}
    # 获取cluster的center
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    # 获取每个cluster的关键特征
    # 获取每个cluster的书
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index] for index in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features
        
        books = book_data[book_data['Cluster'] == cluster_num]['title'].values.tolist()
        cluster_details[cluster_num]['books'] = books
        
    return cluster_details

def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details:'.format(cluster_num))
        print('-' * 20)
        print('Key features:', cluster_details['key_features'])
        print('book in this cluster:')
        print(', '.join(cluster_details['books']))
        print('=' * 40)

cluster_data = get_cluster_data(clustering_obj=km_obj, book_data=book_data,
                               feature_names=feature_names, num_clusters=num_clusters, topn_features=5)
print_cluster_data(cluster_data)

dict_items([(9, 1106), (7, 388), (1, 230), (6, 108), (3, 136), (2, 278), (5, 110), (0, 143), (4, 235), (8, 88)])
Cluster 0 details:
--------------------
Key features: ['中国', '历史', '古代', '一部', '社会']
book in this cluster:
看见, 平凡的世界（全三部）, 平凡的世界（全三部）, 中国历代政治得失, 月光落在左手上, 青铜时代, 东宫·西宫, 佛祖在一号线, 诗词会意：周汝昌评点中华好诗词, 山海经, 傲慢与偏见, 四世同堂, 第一炉香, 怨女, 望春风, 谈艺录, 钱钟书选集·散文卷，小说诗歌卷, 旧文四篇, 傲慢与偏见, 诗词会意：周汝昌评点中华好诗词, 山海经, 中国北方的情人, 松风, 偏爱你的甜, 11处特工皇妃, 笑傲江湖（全四册）, 剑桥简明金庸武侠史, 多情剑客无情剑(上、中、下), 11处特工皇妃, ﻿11处特工皇妃, 新宋, 笑傲江湖（全四册）, 剑桥简明金庸武侠史, 金庸散文集, 松风, 多情剑客无情剑(上、中、下), 万历十五年, 中国历代政治得失, 叫魂, 自控力, 邓小平时代, 乡土中国, ﻿未来简史, 乡土中国, 叫魂, 看见, 乡土中国, 寻路中国, 中国历代政治得失, 邓小平时代, 中国建筑史, 华夏意匠, 图像中国建筑史, 中国古代建筑史, 空谷幽兰, 正见, 金刚经, 八万四千问, 联邦党人文集, 我的前半生, ﻿万历十五年, 中国历代政治得失, 中国近代史, 中国大历史, 中国历代政治得失, 中国文化的深层结构, 正见, 中国哲学简史, 中国古代文化常识, 中国文化要义, 经典里的中国, 未来简史, 中国大历史, 李鸿章传, 硅谷钢铁侠, 我的前半生, 隔江山色, 隔江山色, ﻿金刚经, 正见, 维摩诘经, 布局天下, 活着回来的男人, 中国近代史, 中国1945, 李鸿章传, 中国近代史, 近代中国社会的新陈代谢, 潮来潮去, 档案中的历史, 美术、神话与祭祀, 宗子维城, 中国天文考古学, 中国青铜时代, 唐风吹拂撒马尔罕, 暗流, 何以中国, 中国古代壁画 唐代, 自由与繁荣的国度, 联邦党人文集, 中国美术史